# Seating plan generator

This was made for the [CSS Ball](https://cssbham.com/ball/2023). This approach uses simulated annealing to generate a seating plan for the ball.

## Todo

* Write a better neighbourhood function
* Improve cost function to account for groups split across different tables
* Allow for empty seats / tables, and dynamically sized tables

In [ ]:
import random
import numpy as np
import pandas as pd
import networkx as nx
import copy
import math

## Parameters

In [ ]:
num_tables = 15
table_size = 10

Download the Readable Data spreadsheet as a CSV and save it as `attendees.csv` in the same directory as this notebook.

In [ ]:
guests = []
guest_groups = {}
df = pd.read_csv('attendees.csv')

for index, row in df.iterrows():
    guests.append((row['Formatted Preferred Name'], row['First Seat Preference'], row['Second Seat Preference']))


def find_guest_prefs(guest_name):
    global guests
    guest = [n for n in guests if n[0] == guest_name]
    if len(guest) == 0:
        return None
    return [guest[0][1], guest[0][2]]

def print_arrangement(arrangement):
    for index, table in enumerate(arrangement):
        print(f'Table {index}: {table}')

## Initial arrangement generation

This generates a random arrangement of attendees at tables.

In [ ]:
def random_arrangement(guests):
    random.shuffle(guests)
    arrangement = []
    for i in range(num_tables):
        table = []
        for j in range(table_size):
            if len(guests) == 0:
                break
            last, guests = guests[-1], guests[:-1]
            table.append(last[0])
        arrangement.append(table)

        if len(guests) == 0:
            break

    return arrangement

Alternatively, instead of generating a random arrangement, we can seat groups of attendees at a time.

In [ ]:
G = nx.Graph()
for guest in guests:
    G.add_node(guest[0])
    for pref in guest[1:3]:
        if any(tup[0] == pref for tup in guests):
            G.add_edge(guest[0], pref)

for index, SG in enumerate(nx.connected_components(G)):
    for guest in SG:
        guest_groups[guest] = index

initial = []
current_table = []
for index, SG in enumerate(nx.connected_components(G)):
    for guest in SG:
        if len(current_table) < table_size:
            current_table.append(guest)
        else:
            initial.append(current_table)
            current_table = [guest]
        guest_groups[guest] = index
    initial.append(current_table)
    current_table = []

del current_table

In [ ]:
# initial = random_arrangement(guests.copy())
print_arrangement(initial)


## Cost function

The cost function adds one for each guest that is not seated beside their preferences. 

TODO: This is a very simple cost function, and could be improved to account for groups split across different tables.

In [ ]:
def individual_cost(table, table_index, guest_prefs):
    cost = 0
    if not table[(table_index - 1) % len(table)] in guest_prefs:
        cost += 1
    if not table[(table_index + 1) % len(table)] in guest_prefs:
        cost += 1
    return cost


def cost(seating_arrangement):
    total_cost = 0

    for table in seating_arrangement:
        for i, guest in enumerate(table):
            guest_prefs = find_guest_prefs(guest)
            total_cost += individual_cost(table, i, guest_prefs)
    
    return total_cost

def cost_prime_breakdown(seating_arrangement):
    singular_preference_cost = 0
    mutual_preference_cost = 0
    group_cost = 0

    for table in seating_arrangement:
        for i, guest in enumerate(table):
            guest_prefs = find_guest_prefs(guest)
            adjacent_guests = [table[(i - 1) % len(table)], table[(i + 1) % len(table)]]
            for pref in guest_prefs:
                if pref not in adjacent_guests:
                    pref_prefs = find_guest_prefs(pref)
                    if pref_prefs is not None and guest in pref_prefs:
                        mutual_preference_cost += 10
                    else:
                        singular_preference_cost += 1
            
            group = guest_groups[guest]
            group_size = len([guest for guest in guest_groups if guest_groups[guest] == group])
            if group_size > table_size:
                continue
            elif group_size == 1:
                continue
            group_seated = len([guest for guest in table if guest_groups[guest] == group])
            if group_seated == 1:
                group_cost += 10
            elif group_size - group_seated > 0:
                # group_cost += (group_size - group_seated) / 2 if group_size > 5 else (group_size - group_seated)
                group_cost += 1 * (1/group_size) if group_size > 5 else 3 * (1/group_size)

    
    return singular_preference_cost, mutual_preference_cost, group_cost

def cost_prime(seating_arrangement):
    return sum(cost_prime_breakdown(seating_arrangement))


In [ ]:
print(f'Cost of initial arrangement: {cost_prime(initial)}')

## Neighbourhood function

The neighbourhood function swaps two guests at random. It is shit.

In [ ]:
def neighbour_random(seating_arrangement):
    seating_arrangement = copy.deepcopy(seating_arrangement)
    tables = random.sample(range(len(seating_arrangement)), len(seating_arrangement))
    table_from = tables.pop()
    while len(seating_arrangement[table_from]) == 0:
        table_from = tables.pop()
    seat_from = random.sample(range(len(seating_arrangement[table_from])), len(seating_arrangement[table_from])).pop()
    table_to = tables.pop()

    if len(seating_arrangement[table_to]) < table_size:
        seating_arrangement[table_to].append(seating_arrangement[table_from][seat_from])
        seating_arrangement[table_from].pop(seat_from)
    else:
        seat_to = random.sample(range(len(seating_arrangement[table_to])), len(seating_arrangement[table_to])).pop()
        to = seating_arrangement[table_to][seat_to]
        seating_arrangement[table_to][seat_to] = seating_arrangement[table_from][seat_from]
        seating_arrangement[table_from][seat_from] = to
    
    seating_arrangement = list(filter(lambda x: len(x) > 0, seating_arrangement))
    seating_arrangement.append([])

    return seating_arrangement

## Simulated annealing

Main loop for simulated annealing. Takes cost function and neighbourhood function as parameters.

In [ ]:
def simulated_annealing(initial, cost, neighbour, temperature, cooling_rate, stopping_temperature, min_iters=0):
    current_state = initial
    current_cost = cost(current_state)
    best_state = current_state
    best_cost = current_cost
    current_iters = 0
    
    while temperature > stopping_temperature or current_iters < min_iters:
        new_state = neighbour(current_state)
        new_cost = cost(new_state)
        delta = current_cost - new_cost
        
        if delta > 0:
            current_state = new_state
            current_cost = new_cost
            if current_cost < best_cost:
                best_state = current_state
                best_cost = current_cost
        else:
            probability = math.exp(delta / temperature)
            if random.random() < probability:
                current_state = new_state
                current_cost = new_cost
        
        temperature *= cooling_rate
        current_iters += 1
    
    return best_state, best_cost

Run the actual simulation.

In [ ]:
best_seating_arrangement, best_cost = simulated_annealing(initial, cost_prime, neighbour_random, temperature=1000000, cooling_rate=0.99, stopping_temperature=0.00001)

print_arrangement(best_seating_arrangement)
singular_preference_cost, mutual_preference_cost, group_cost = cost_prime_breakdown(best_seating_arrangement)
print(f'Cost')
print(f'   Mutual pref: +{mutual_preference_cost}')
print(f'   Preferences: +{singular_preference_cost}')
print(f'  Split groups: +{group_cost}')
print(f'                ={best_cost}')